# Annika Carlson
## AEEM5042 - Module 4 Assignment

In [20]:
import numpy as np
from IPython.display import display, Markdown

### Question 1

Set given/known values:

In [ ]:
M0 = 0.8        # freestream mach number
A0_A2 = 0.92    # capture to engine face area ratio

Initial Assumptions:

In [22]:
y = 1.4        # ratio of specific heats, cold

Find:

a) Pressure coefficient, $C_p$, at the stagnation point

b) Inlet Mach number, $M_1$

c) Lip contraction ratio, $A_1/A_{th}$, for a throat Mach $M_{th}$ = 0.75, assuming $p_t/p_{th}$ = 1

d) Diffuser area ratio, $A_2/A_{th}$ if $M_2$ = 0.5 and $p_t/p_{th}$ = 0.98

e) The non-dimensional inlet additive drag, $D_{add}/\left( p_0 \cdot A_1 \right)$

#### a)

Start by getting the equation for pressure coefficient, $C_p$, in terms of freestream mach number using pressure isentropic relations.

$$
C_p = \frac{2\left(P_{t0} - P_0\right)}{\gamma \cdot P_0 \cdot M_0^2}
$$

where $P_{t0}$ is freestream total pressure, and $P_0$ is freestream static pressure. From isentropic relations:

$$
\frac{P_{t0}}{P_0} = \left(1 + \frac{\gamma - 1}{2}*M_0^2\right)^{\frac{\gamma}{\gamma - 1}} \Rightarrow P_{t0} = P_0 \left(1 + \frac{\gamma - 1}{2}*M_0^2\right)^{\frac{\gamma}{\gamma - 1}}
$$

Plugging this into the $C_p$ equation for $P_{t0}$ and simplifying, we get:

$$
C_p = \frac{2\left[P_0 \left(1 + \frac{\gamma - 1}{2}*M_0^2\right)^{\frac{\gamma}{\gamma - 1}} - P_0\right]}{\gamma \cdot P_0 \cdot M_0^2} = \frac{2 \cdot P_0\left[\left(1 + \frac{\gamma - 1}{2}*M_0^2\right)^{\frac{\gamma}{\gamma - 1}} - 1\right]}{\gamma \cdot P_0 \cdot M_0^2}
$$

$$
\Rightarrow C_p = \frac{2 \left[\left(1 + \frac{\gamma - 1}{2}*M_0^2\right)^{\frac{\gamma}{\gamma - 1}} - 1\right]}{\gamma \cdot M_0^2} 
$$

This leaves us $C_p$ in terms of $\gamma$ and $M_0$, which we can now solve.

In [23]:
Cp = (2*((1 + ((y - 1)/2)*(M0**2))**(y/(y - 1)) - 1))/(y*(M0**2))
display(Markdown(f"$C_p$ = ${Cp:.2f}$"))

$C_p$ = $1.17$

#### b)

#### c)

Assume:

In [ ]:
Mth = 0.75      # throat Mach number
Pt2_Pth = 1

#### d)

#### e)

### Question 2

Set given/known values:

Assumptions:

ADD

Find:

Capture-to-engine face area ratio, $A_0/A_2$

### Question 3

Set given/known values:

Assumptions:

ADD

Find:

a) Inlet Mach number, $M_1$

b) Inlet total pressure recovery, $\pi_d = P_{t2}/P_{t0}$

#### a)

#### b)

### Question 4

Set given/known values:

Assumptions:

ADD

Find:

a) Total pressure recovery of the inlet shock system, $\pi_d = P_{t2}/P_{t0}$

b) Compare the total pressure recovery to a normal shock (only) inlet at the same Mach number

#### a)

#### b)

### Question 5

Set given/known values:

Assumptions:

ADD

Find:

a) Ambient pressure, $P_0$ $\quad$ Note: $\left(P_0 = P_y\right)$

b) Exhaust temperature, $T_y$

c) Mass flow rate through the nozzle

#### a)

#### b)

#### c)

### Question 6

Set given/known values:

Assumptions:

ADD

Find:

All dimensions dimensions including throat diameter ($D_8$), exit diameter ($D_9$), divergence angle ($\alpha$), length of divergent section ($L_s$) 

and $C_{fg}$, $F_g$, $C_V$